In [1]:
import pandas as pd
import numpy as np
import os

# --- CONFIGURATION ---
DATA_PATH = '/home/delaunan/code/delaunan/clintrialpredict/data'
MAIN_DATA_FILE = 'project_data.csv'      # Your existing processed data
EMB_FILE = 'embeddings_with_nctid.csv'   # The new file to merge

def audit_embeddings():
    print("=== 🔍 CLINICAL TRIAL DATA AUDIT: EMBEDDINGS INTEGRATION ===\n")

    # 1. Load Paths
    path_main = os.path.join(DATA_PATH, MAIN_DATA_FILE)
    path_emb = os.path.join(DATA_PATH, EMB_FILE)

    # 2. Load Main Data
    if os.path.exists(path_main):
        print(f"1. Loading Main Cohort: {MAIN_DATA_FILE}...")
        df_main = pd.read_csv(path_main, usecols=['nct_id', 'overall_status', 'phase'])
        print(f"   -> Rows: {len(df_main):,}")
        print(f"   -> Unique NCT_IDs: {df_main['nct_id'].nunique():,}")
    else:
        print(f"   [!] CRITICAL: {MAIN_DATA_FILE} not found. Please generate it first.")
        return

    # 3. Load Embeddings
    if os.path.exists(path_emb):
        print(f"\n2. Loading Embeddings: {EMB_FILE}...")
        try:
            df_emb = pd.read_csv(path_emb)
            print(f"   -> Rows: {len(df_emb):,}")
            print(f"   -> Columns: {len(df_emb.columns)}")
        except Exception as e:
            print(f"   [!] Error reading embedding file: {e}")
            return
    else:
        print(f"   [!] CRITICAL: {EMB_FILE} not found.")
        return

    # 4. Check Embedding File Integrity
    print("\n3. Checking Embedding Integrity...")

    # Check for Duplicate NCT_IDs in embeddings
    if 'nct_id' not in df_emb.columns:
        print("   [!] ERROR: 'nct_id' column missing in embeddings file.")
        return

    dupes = df_emb['nct_id'].duplicated().sum()
    if dupes > 0:
        print(f"   [!] WARNING: Found {dupes} duplicate NCT_IDs in embedding file. First occurrence will be kept.")
    else:
        print("   [✓] No duplicate NCT_IDs in embedding file.")

    # Check for NaNs in embedding vectors
    emb_cols = [c for c in df_emb.columns if c != 'nct_id']
    print(f"   -> Detected {len(emb_cols)} embedding dimensions.")

    nans = df_emb[emb_cols].isna().sum().sum()
    if nans > 0:
        print(f"   [!] WARNING: Found {nans} NaN values in embedding columns. They will be filled with 0.")
    else:
        print("   [✓] No NaNs in embedding vectors.")

    # 5. MERGE SIMULATION & STATS
    print("\n4. Merge Simulation (Left Join)...")

    # Identify keys
    main_ids = set(df_main['nct_id'])
    emb_ids = set(df_emb['nct_id'])

    # Intersection
    common = main_ids.intersection(emb_ids)
    missing = main_ids - emb_ids
    unused = emb_ids - main_ids

    print(f"   --------------------------------------------------")
    print(f"   Total Trials in Main Cohort:  {len(main_ids):>8,}")
    print(f"   --------------------------------------------------")
    print(f"   ✅ MATCHED (Have Embeddings):  {len(common):>8,}  ({len(common)/len(main_ids):.1%})")
    print(f"   ⚠️ MISSING (Will be 0-filled): {len(missing):>8,}  ({len(missing)/len(main_ids):.1%})")
    print(f"   --------------------------------------------------")
    print(f"   ℹ️  Unused Embeddings (ignored):{len(unused):>8,}")

    # 6. Drill Down on Missing Data (If any)
    if len(missing) > 0:
        print(f"\n5. Analysis of MISSING Embeddings (The {len(missing)/len(main_ids):.1%})...")
        df_missing = df_main[df_main['nct_id'].isin(missing)]

        print("   Missing by Status:")
        print(df_missing['overall_status'].value_counts(normalize=True).to_string(float_format="{:.1%}".format))

        print("\n   Missing by Phase:")
        print(df_missing['phase'].value_counts(normalize=True).to_string(float_format="{:.1%}".format))

        print("\n   [Interpretation]")
        if len(missing) < 0.1 * len(main_ids):
            print("   -> Missing rate is LOW (<10%). Filling with 0.0 is safe.")
        else:
            print("   -> [!] Missing rate is HIGH (>10%). Ensure your embedding generation covered the full date range (2000-2015).")

# Run the audit
audit_embeddings()

=== 🔍 CLINICAL TRIAL DATA AUDIT: EMBEDDINGS INTEGRATION ===

1. Loading Main Cohort: project_data.csv...
   -> Rows: 67,363
   -> Unique NCT_IDs: 67,363

2. Loading Embeddings: embeddings_with_nctid.csv...
   -> Rows: 105,336
   -> Columns: 101

3. Checking Embedding Integrity...
   [✓] No duplicate NCT_IDs in embedding file.
   -> Detected 100 embedding dimensions.
   [✓] No NaNs in embedding vectors.

4. Merge Simulation (Left Join)...
   --------------------------------------------------
   Total Trials in Main Cohort:    67,363
   --------------------------------------------------
   ✅ MATCHED (Have Embeddings):    67,363  (100.0%)
   ⚠️ MISSING (Will be 0-filled):        0  (0.0%)
   --------------------------------------------------
   ℹ️  Unused Embeddings (ignored):  37,973
